# Daten
1. Regen - Ja, Nein
2. Niederschlagsmenge
3. Klarer himmel - Ja, Nein
4. Temperatur
5. Windgeschwindigkeit

API call - https://api.darksky.net/forecast/[key]/[latitude],[longitude],[time]

In [178]:
import datetime
import requests
import pandas as pd
import os

In [3]:
def date_range_gen(start, end, delta):
    """
        input:

        start: date obj
        end: date obj
        delta: int
    """
    
    dates = []
    while start <= end:
        dates.append(start)
        start = start + datetime.timedelta(days=delta)
    return dates

In [182]:
def api_string_gen(api_key, latitude, longtitude,
                   date_start, date_end, date_delta, 
                   time_start, time_end, time_delta, gmt):
    """
    input:
        api_key: str
        latitude: str
        longtitude: string
        date_start: date obj
        date_end: date obj
        time_start: int (in hours)
        time_end: int (in hours)
        time_delta: int (in hours)
        gmt: string in format hh:mm
    returns:
        api_url: str
        date: str
        time: str
    """
    url = r'https://api.darksky.net/forecast/'
    
    date_range = date_range_gen(date_start, date_end, date_delta)
    time_range = range(time_start, time_end)
    
    for date in date_range:
        for time in time_range:
            yield url + api_key + \
            r'/' + latitude + ',' + longtitude + ',' + str(date) + "T" + '%02d'%time + ":00:00+" + gmt + \
            '?units=auto', str(date), '%02d'%time

In [179]:
def download_data(filepath, api_keys,       
                  latitude, longtitude,
                  date_start, date_end, date_delta, 
                  time_start, time_end, time_delta, gmt):
    """
        The method goes through all api keys. If the calls are exceeded, the next API key is taken. If there are no more
        API Keys left, the data is saved. The format is from_date_time-to_date_time.csv
    """
    api_key = api_keys.pop()
    
    # Generate API strings
    strings_gen = api_string_gen(api_key, latitude, longtitude,
                                 date_start, date_end, date_delta, 
                                 time_start, time_end, time_delta, gmt)
    s = strings_gen.next()
    date_start, time_start = s[1], s[2]
    
    try:
        # Call the API 
        r = requests.get(s[0])
        
        # Create the DataFrame
        df_complete = pd.DataFrame(r.json())['currently'].to_frame()
        df_complete['data'] = df_complete.index
        df_complete['date'] = str(s[1]) + 'T' + s[2] + ':00:00'
        df_complete = df_complete.pivot(columns = 'data', values='currently', index='date')
        
    except ValueError:
        print "The API calls for %s are exceeded. Taking the next API. There are %s API keys left" % (api_key, len(api_keys))
        try:
            api_key = api_keys.pop()
        except IndexError:
            print "There are no more API keys"
            return
    
    # Append to the DataFrame
    for s in strings_gen:
        try:
            # Call the API 
            r = requests.get(s[0])
            
            # Create, transform and append the DataFrame
            df = pd.DataFrame(r.json())['currently'].to_frame()
            df['data'] = df.index
            df['date'] = str(s[1]) + 'T' + s[2] + ':00:00'
            df = df.pivot(columns = 'data', values='currently', index='date')
            df_complete = df_complete.append(df)
        except ValueError:
            print "The API calls for %s are exceeded. Taking the next API. There are %s API keys left" % (api_key, len(api_keys))
            try:
                api_key = api_keys.pop()
            except IndexError:
                print "There are no more API keys"
                date_end, time_end = s[1], s[2]
                df_complete.to_csv(os.path.join(filepath, date_start + "T" + time_start + "-" +
                                                date_end + 'T' + time_end + '.csv'))
                return df_complete
        
    
    date_end, time_end = s[1], s[2]
    df_complete.to_csv(os.path.join(filepath, date_start + "T" + time_start + "-" +
                                    date_end + 'T' + time_end + '.csv'))
    return df_complete

In [188]:
# Parameters
filepath = r"C:\Data\wetter"

api_emails = ['13obo94@gmail.com', 'donchev.boris@gmail.com', 'martin.yankov@gmail.com']
api_keys = ['012bd47ef29c78d41d802935d731d876', '5fa4d2ae76bc4035201e84ac60e06f8e', '797eba2745e8f69147ad6b4f8bacca29'] 

beginning = '2017-10-17'
end = '2017-11-15'
d_delta = 1
hours_start = 0
hours_end = 24
h_delta = 1

latitude = '51.960665'
longtitude = '7.626135'

date_beginning = datetime.datetime.strptime(beginning, '%Y-%m-%d').date()
date_end = datetime.datetime.strptime(end, '%Y-%m-%d').date()

___

In [189]:
df = download_data(filepath, api_keys,
                   latitude, longtitude,
                   date_beginning, date_end, d_delta,
                   hours_start, hours_end, h_delta, '01:00')

The API calls for 797eba2745e8f69147ad6b4f8bacca29 are exceeded. Taking the next API. There are 2 API keys left
The API calls for 5fa4d2ae76bc4035201e84ac60e06f8e are exceeded. Taking the next API. There are 1 API keys left
The API calls for 012bd47ef29c78d41d802935d731d876 are exceeded. Taking the next API. There are 0 API keys left
There are no more API keys
